# The Traveling Salesperson Problem
If you have never encountered the traveling sales person problem, it can be seen <a href="https://en.wikipedia.org/wiki/Travelling_salesman_problem">here</a>. It is a classic problem in the field of computer science, and most certainly worth looking. A short and sweet problem description is as follows:
> *Given a set of cities and the distance between each pair of cities, what is the shortest possible tour that visits each city exactly once, and returns to the starting city?*

This walkthrough is going to work on developing some solutions to the problem, and in general demonstrate *how to think about* solving problems. 

<img src="images/tsp.png">

The first thing we want to ask whenever faced with a problem of this sort, is do we understand the problem statement enoguh to program a solution? Let's think for a moment:
<br>
* ***Given a set of cities***<br>
    A Python `set` could represent a set of cities. An individual city may be just an integer index, or it might be (x,y) coordinates.  
* ***...and the distance between each pair of cities***<br>
    We could use either a function, `distance(A, B)`, or a table, `distance[A,B]`. 
* ***...what is the shortest possible tour***<br>
    A tour is a sequential order in which to visit the cities; a function `shortest_tour(tours)` should find the one that minimizes `tour_length(tour)`, which is the sum of the distances between adjacent cities in the tour. 
* ***...that visits each city once and returns to the starting city***<br>
    Make sure the tour doesn't re-visit a city (except returning to the start)
    
At this point we don't have all of the answers, but we have are at a good point to start attacking the problem! We can being by bringing in a few imports that we will utilize during the notebook.

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
from time import clock
from itertools import permutations, combinations
from functools import lru_cache as cache
from collections import Counter
from statistics import mean, median

---
# Exhaustive Search Algorithm: `exhaustive_tsp`
To begin, let's start with an algorithm that comes with a *guarantee* that it will solve the problem, albeit inefficiently. 
> **Exhaustive Search Algorithm**: *Generate all possible tours of the cities, and choose the shortest tour (the one with minimum tour length).*

The design philosphy that we will follow here is to first write an English description of the algorithm (as we did above), and then write python code that closely mirrors the English description. This will probably require some auxilliary functions and data structures; at first we can just assume that they exist, put them on a TO DO list, and eventually define them with the same design philosophy. 

In [31]:
def exhaustive_tsp(cities):
  """Generate all possible tours of the cities and choose the shortest tour."""
  return shortest_tour(alltours(cities))

def shortest_tour(tours): return min(tours, key=tour_length)

# TO DO: Data types: City, Cities, Tour; Functions: alltours, tour_length

This gives us a good starting point. The Python code above closely matches our English description. Now we can tackle the TO DO list. 

**Tours**: A tour that starts in city 1, moves to 2, then 3, then back to 1 will be represented by `(1, 2, 3)`. Any valid tour of a set of cities will be a *permutation* of the cities. That means that we can implement `alltours` with the built in `permutations` function (from `itertools` module). Recall, **permutation** is the case where order matters. The length of a tour is the sum of the distances between adjacent cities in the tour - the sum of the lengths of the **links** between cities in the tour. 

**Cities**: The only thing we need to know about a city is its distance to other cities. We don't need to know its name, population, or anything else. We'll assume the distance between two cities is the <a href="https://en.wikipedia.org/wiki/Euclidean_distance">Euclidean distance</a>, the straight-line distance between points in a two-dimensional plane. So we want `City(300, 100)` to be the city with x-coordinate 300 and y coordinate 100. At first glance it seems like Python does not have a builtin type for a point in the two-dimensional plane, but there actually is one: complex numbers. We can implement `City` with `complex`, which means the distance between two cities, `distance(A, B)` is the absolute value of the vector difference between them. If has been a while since you worked with complex numbers, recall that the absolute value of a complex number is defined as:

#### $$|a + bi| = \sqrt{a^2 + b^2}$$

The absolute value of a complex number is also known as the **modulus**. In case this isn't clear, recall that the definition of the absolute value of a variable, $x$, is defined as:

#### $$|x| = \sqrt{x^2}$$

And we can apply the geometric interpretation of the absolute value of a real number, *it's distance from the origin 0*, to the complex case. Hence, the absolute value of a complex number is defined by the Euclidean distance of the complex point to the origin. The can be computed using pythagorean theorem, but thankfully python has a built in `abs` function we can make use of. Feel free to read more about <a href="https://en.wikipedia.org/wiki/Absolute_value#Complex_numbers">absolute value of complex numbers</a>.

We can also define `Cities(n)` to make a set of `n` random cities. We want `Cities(n)` to be reproducible (to return the same result when called with the same arugments), so we provide an optional argument that sets `random.seed`. 

In [32]:
alltours = permutations                   # Setting alltours to be equivalent to built in permutations function

def tour_length(tour):
  """The total of distances between each pair of consecutive cities in the tour.
  This includes the last-to-first, distance(tour[-1, tour[0]])"""
  return sum(distance(tour[i - 1], tour[i]) for i in range(len(tour)))

City = complex                                         # Initialize City to be a complex variable

def distance(A, B): return abs(A - B)                  # Return absolute value of complex variable

def Cities(n, seed=123, width=999, height=666):
  """Create a set of n cities, sampled uniformly from a (width x height) rectangle"""
  random.seed((n, seed))
  return frozenset(City(random.randint(1, width), random.randint(1, height)) for c in range(n))

# Our first solution!
At this point we have built all of the functions that we need. We are ready to use `exhaustive_tsp` to find a tour for a set of cities:

In [34]:
exhaustive_tsp(Cities(9))

((158+421j),
 (297+397j),
 (832+102j),
 (872+207j),
 (817+315j),
 (939+600j),
 (620+498j),
 (163+639j),
 (31+501j))

You may immediately be asking yourself, but how do we know that this is the shortest tour? Let's create set of functions that allows us to visualize our results. 

# Visualizing results: `plot_tour` and `do` 

In [ ]:
def plot_tour(tour, style='bo-'):
  """Plot every city and link in the tour, and highlight start city."""
  if len(tour) > 1000: plt.figure(figsize=(15,10))
  start = tour[0:1]
  plot_segment(tour + start, style)
  plot_segment(start, 'rD')                # The start city will be a red diamond